# GoTo Scope

One of the first pieces of functionality empowered by the automation of the 2 axis of the Dobsonian telescope is the ability to go to a specific position in the skys. The exact positions of stellar objects are well documented. A very common coordinate framework is based upon the polar coordinates; RA/Dec (Right Ascension/Declination) in degrees.

But to use RA/Dec it must be converted to AltAz (Altitude/Azimuth). The AltAz coordinate system is a more convenient system for the telescope to use. The AltAz coordinate system is defined by the telescope's location on the sky, and the location of the object in the sky. The AltAz is based on the polar coordinates that each of the axis of automation enable. Azimuth is the angle to the right of the North (compass heading), and Altitude is the angle above the horizon.

Our motorized telescope is limited to its degrees of motion. 

The Altitude goes from 0 degrees, straight up, to hopefully 90 degrees, horizontal to the horizon. Thee reality is your viewing site limits how close to horizontal you can achieve. In addition the strength of the motor that controls this axis also limits how far you can go if the telescope is not well balanced, there tends to me weight out near the moment arm of the telescope; cameras, secondary mirror ect.

The Azimuth is limited because the its motion is limited because the drive belts and the angle to the motor. It is limited to approximately 270 degrees of rotation so that the angle of incidence of the belt to the rotating table does not change thus effecting the belts tension.  A good range would be 250 degrees, + or - 125 degrees from center. So because of this it is best to align the assumed center by pointing the telescope at a well known compass heading when at center. For now center is recognized when the belt tensioner is directly opposite the driver motor. You can check this by feeling under the telescope for its position.  This could be improved by a sensor or some visual clue.

This limitation of Azimuth is acceptable because the telescope can physically be repositioned to view other parts of the sky of interest and does not interfere with tracking. There can also be site based limitations based on physical structures and the surrounding environment, (house, open sky due to trees, etc.).


This leads us to needing to convert the RA/Dec to an exact number of motor steps to move the scope to (via AltAz) in order to position the object in the telescopes Field of View.

To help facilitate this I turn to [AstroPy](https://docs.astropy.org/en/stable/index.html). a Python library for Astronomy and Astrophysics. It is a great tool for working with coordinate systems.

## Setup needed libraries

In [62]:
%pip install astropy
%pip install pytz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [11]:

from astropy.coordinates import EarthLocation, SkyCoord, AltAz
from astropy.time import Time
from datetime import datetime, tzinfo
from pytz import timezone
from typing import Dict, Any, Tuple, Union

observatory_parameters: Dict[str, Union[str,float]] = { 
    'timezone' : 'America/Chicago', 
    'location' : '106 Melody Cir Verona WI',
    'az_heading_degrees': 180.0,
    'alt_steps_per_degree' : 1725000 / 90.0,
    'az_steps_per_degree' : 200000/ 90.0

    }

class Observatory:
    _observatory_parameters: Dict[str, Union[str,float]] = None
    _local_time_tz : tzinfo
    _observatory : EarthLocation

    def __init__(self, observatory_parameters: Dict[str, Union[str,float]]) -> EarthLocation:
        self._observatory_parameters = observatory_parameters
        self._local_time_tz = timezone(observatory_parameters['timezone'])
        self._observatory = EarthLocation.of_address(observatory_parameters['location'])
        
    def _get_alt_az_in_motor_steps(self, target_coordinates: SkyCoord, target_time: str) -> Tuple[int,int]:
        local_dt = self._local_time_tz.localize(datetime.strptime(target_time, "%Y-%m-%d %H:%M:%S"))

        alt_az : AltAz = AltAz(location=self._observatory, obstime=Time(local_dt))
        target_alt_az : Any = target_coordinates.transform_to(alt_az)

        alt_motor_steps = int((90.0 - target_alt_az.alt.degree) * self._observatory_parameters['alt_steps_per_degree'])
        az_motor_steps = int((self._observatory_parameters['az_heading_degrees'] - target_alt_az.alt.degree) * self._observatory_parameters['az_steps_per_degree'])

        return alt_motor_steps, az_motor_steps

    # target_time is in format "2001-1-2 20:00:00"
    def goto(self, target_coordinates: SkyCoord, target_time: datetime ) -> None:
        alt_motor_steps, az_motor_steps = self._get_alt_az_in_motor_steps(target_coordinates, target_time)

        # move alt motor
        print(alt_motor_steps)

        # move az motor
        print(az_motor_steps)


observatory : Observatory = Observatory(observatory_parameters)


In [12]:
Rigel = SkyCoord('5h15m35.01s', '-8d10m37.1s')
observatory.goto(Rigel, '2001-1-2 20:00:00')

Betelgeuse = SkyCoord('5h56m20.95', '7d24m37.8s')
observatory.goto(Betelgeuse, '2001-1-2 20:00:00')

1161049
334614
1027816
319167
